In [40]:
import numpy as np 
import pandas as pd 
# import geopandas as gpd
from tqdm import tqdm
# from tensorflow.keras import models

data = pd.read_csv("train.csv")
data_size = len(data['Id'])
data["Country_Region"] = [country_name.replace("'","") for country_name in data["Country_Region"]]
data = data.query("Date>'2020-01-29' and Date<='2020-03-11'")
trend_df = pd.DataFrame(columns={"infection_trend","fatality_trend","expected_cases","expected_fatalities"})

# for country in data.Country_Region.unique():
#     print(country)

    # count = 0
    # for province in data.query("Country_Region=='"+country+"'").Province_State:
    #     count+=1

    # print(count)
tensor_size = 7;
with tqdm(total=len(data.Country_Region.unique())) as pbar:
    for country in data.Country_Region.unique():
        for province in data.query("Country_Region=='"+country+"'").Province_State:
            province_df = data.query("Country_Region=='"+country+"' and Province_State=='"+str(province)+"'")
            for i in range(0,len(province_df)-tensor_size,tensor_size):

                infection_trend = [float(x) for x in province_df[i:i+tensor_size-1].ConfirmedCases.values]
                fatality_trend = [float(x) for x in province_df[i:i+tensor_size-1].Fatalities.values]
                expected_cases = float(province_df.iloc[i+tensor_size-1].ConfirmedCases)
                expected_fatalities = float(province_df.iloc[i+tensor_size-1].Fatalities)
                                            
                trend_df = trend_df.append({"infection_trend":infection_trend,
                                 "fatality_trend":fatality_trend,
                                 "expected_cases":expected_cases,
                                 "expected_fatalities":expected_fatalities},ignore_index=True)
        pbar.update(1)

100%|██████████| 173/173 [02:41<00:00,  1.07it/s]


In [41]:
# print(trend_df)
trend_df["input"] = [np.asarray([np.array(trends["infection_trend"]),np.asarray(trends["fatality_trend"])]) for idx,trends in trend_df.iterrows()]
print(trend_df)

infection_trend  expected_fatalities  \
0      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                  0.0   
1      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                  0.0   
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                  0.0   
3      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                  0.0   
4      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                  0.0   
...                               ...                  ...   
26245  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                  0.0   
26246  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                  0.0   
26247  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                  0.0   
26248  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                  0.0   
26249  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]                  0.0   

                       fatality_trend  expected_cases  \
0      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0   
1      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0   
2      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]             0.0   
3      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]            

In [52]:
from sklearn.utils import shuffle
trend_df = shuffle(trend_df)
trend_df.head()


,infection_trend,expected_fatalities,fatality_trend,expected_cases,input
6515,"[4903.0, 5806.0, 7153.0, 11177.0, 13522.0, 166...",549.0,"[162.0, 204.0, 249.0, 350.0, 414.0, 479.0]",19665.0,"[[4903.0, 5806.0, 7153.0, 11177.0, 13522.0, 16..."
9761,"[79.0, 81.0, 88.0, 91.0, 95.0, 106.0]",2.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 2.0]",112.0,"[[79.0, 81.0, 88.0, 91.0, 95.0, 106.0], [1.0, ..."
10469,"[174.0, 174.0, 174.0, 174.0, 174.0, 174.0]",2.0,"[2.0, 2.0, 2.0, 2.0, 2.0, 2.0]",174.0,"[[174.0, 174.0, 174.0, 174.0, 174.0, 174.0], [..."
12093,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0...."
10689,"[1205.0, 1205.0, 1205.0, 1205.0, 1206.0, 1213.0]",1.0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",1213.0,"[[1205.0, 1205.0, 1205.0, 1205.0, 1206.0, 1213..."
